In [1]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Data Cleaning").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/14 00:57:07 INFO SparkEnv: Registering MapOutputTracker
24/12/14 00:57:07 INFO SparkEnv: Registering BlockManagerMaster
24/12/14 00:57:07 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/14 00:57:07 INFO SparkEnv: Registering OutputCommitCoordinator


In [2]:
# load data
data_path = "gs://my-bigdatatech-project-jl/landing/itineraries.csv"
df_cleaned = spark.read.csv(data_path, header=True, inferSchema=True)
df_cleaned.show(5)


24/12/14 00:57:38 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/12/14 00:57:53 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/12/14 00:58:08 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/12/14 00:58:23 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/12/14 00:58:38 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/12/14 00:58:53 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

+--------------------+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+---------------------------------+------------------------+-------------------------------+----------------------+--------------------------+----------------------------+-------------------+-------------------+----------------------------+-------------------------+----------------+-----------------+
|               legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode|segmentsAirlineName|segmentsAirlineCode|segmentsEquipmentDescription|segmentsDurationInSeconds|segmentsDi

In [3]:
# group by all columns and count occurrences
duplicates = df_cleaned.groupBy(df_cleaned.columns).count().filter("count > 1")
duplicates.show()


+-----+----------+----------+---------------+------------------+-------------+--------------+-----------+--------------+------------+---------+--------+---------+--------------+-------------------+---------------------------------+------------------------+-------------------------------+----------------------+--------------------------+----------------------------+-------------------+-------------------+----------------------------+-------------------------+----------------+-----------------+-----+
|legId|searchDate|flightDate|startingAirport|destinationAirport|fareBasisCode|travelDuration|elapsedDays|isBasicEconomy|isRefundable|isNonStop|baseFare|totalFare|seatsRemaining|totalTravelDistance|segmentsDepartureTimeEpochSeconds|segmentsDepartureTimeRaw|segmentsArrivalTimeEpochSeconds|segmentsArrivalTimeRaw|segmentsArrivalAirportCode|segmentsDepartureAirportCode|segmentsAirlineName|segmentsAirlineCode|segmentsEquipmentDescription|segmentsDurationInSeconds|segmentsDistance|segmentsCabinCode

In [4]:

from pyspark.sql.functions import when, col

# count nulls in a column
def count_nulls(df, column_name):
    return df.select(
        when(col(column_name).isNull(), 1).alias(column_name)
    ).groupBy().sum(column_name).collect()[0][0]

null_count = count_nulls(df_cleaned, "segmentsEquipmentDescription")
print(f"Number of null values in segmentsEquipmentDescription: {null_count}")


Number of null values in segmentsEquipmentDescription: 883152


In [5]:
# impute missing values
# impute `totalTravelDistance` with mean

from pyspark.sql.functions import mean

mean_value = df_cleaned.select(mean("totalTravelDistance")).collect()[0][0]
df_cleaned = df_cleaned.fillna({"totalTravelDistance": mean_value})


In [6]:
# impute `segmentsEquipmentDescription` with mode

mode_value = df_cleaned.groupBy("segmentsEquipmentDescription").count()     .orderBy("count", ascending=False).first()[0]
df_cleaned = df_cleaned.fillna({"segmentsEquipmentDescription": mode_value})

df_cleaned.select("totalTravelDistance", "segmentsEquipmentDescription").show()


+-------------------+----------------------------+
|totalTravelDistance|segmentsEquipmentDescription|
+-------------------+----------------------------+
|                947|                 Airbus A321|
|                947|                 Airbus A321|
|                947|              Boeing 757-200|
|                947|                 Airbus A321|
|                947|                 Airbus A321|
|                947|              Boeing 737-800|
|                956|        Airbus A320||Airb...|
|                956|        Airbus A320||Boei...|
|                956|        Airbus A319||Airb...|
|                956|        Airbus A319||Boei...|
|                947|        Airbus A319||Boei...|
|               1462|        Embraer 175||Boei...|
|               1462|        Embraer 175||Boei...|
|               1462|        Embraer 175||Airb...|
|               1462|        Embraer 175 (Enha...|
|               1571|        AIRBUS INDUSTRIE ...|
|                947|          

In [7]:
# calculate the percentage of missing values for each column

from pyspark.sql import functions as F

total_count = df_cleaned.count()
missing_percentages = {
    col: (df_cleaned.filter(F.col(col).isNull()).count() / total_count) * 100
    for col in df_cleaned.columns
}
# print the missing percentages

print("Missing percentages per column:")
for col, percent in missing_percentages.items():
    print(f"{col}: {percent:.2f}%")


Missing percentages per column:
legId: 0.00%
searchDate: 0.00%
flightDate: 0.00%
startingAirport: 0.00%
destinationAirport: 0.00%
fareBasisCode: 0.00%
travelDuration: 0.00%
elapsedDays: 0.00%
isBasicEconomy: 0.00%
isRefundable: 0.00%
isNonStop: 0.00%
baseFare: 0.00%
totalFare: 0.00%
seatsRemaining: 0.00%
totalTravelDistance: 0.00%
segmentsDepartureTimeEpochSeconds: 0.00%
segmentsDepartureTimeRaw: 0.00%
segmentsArrivalTimeEpochSeconds: 0.00%
segmentsArrivalTimeRaw: 0.00%
segmentsArrivalAirportCode: 0.00%
segmentsDepartureAirportCode: 0.00%
segmentsAirlineName: 0.00%
segmentsAirlineCode: 0.00%
segmentsEquipmentDescription: 0.00%
segmentsDurationInSeconds: 0.00%
segmentsDistance: 0.00%
segmentsCabinCode: 0.00%


In [8]:
# obtain the median value for `totalTravelDistance`
# impute missing values with the median

median_totalTravelDistance = df_cleaned.approxQuantile("totalTravelDistance", [0.5], 0.01)[0]
df_cleaned = df_cleaned.fillna({"totalTravelDistance": median_totalTravelDistance})

# check nulls in `totalTravelDistance`

null_totalTravelDistance = df_cleaned.filter(df_cleaned.totalTravelDistance.isNull()).count()
null_percentage = (null_totalTravelDistance / total_count) * 100
print(f"Percentage of null values in totalTravelDistance: {null_percentage:.2f}%")


Percentage of null values in totalTravelDistance: 0.00%


In [9]:
# drop columns not in human-legible formats

df_cleaned = df_cleaned.drop('segmentsDepartureTimeEpochSeconds', 'segmentsArrivalTimeEpochSeconds')


In [12]:

cleaned_data_path = "gs://my-bigdatatech-project-jl/cleaned/Data_Cleaned.parquet"
df_cleaned.write.mode("overwrite").parquet(cleaned_data_path)
